In [2]:
import os
import json
import numpy as np
from PIL import Image
from pycocotools import mask

def binary_mask_to_rle(binary_mask):
    """Converts a binary mask to RLE format."""
    binary_mask_encoded = mask.encode(np.asfortranarray(binary_mask.astype(np.uint8)))
    binary_mask_encoded['counts'] = binary_mask_encoded['counts'].decode('ascii')
    return binary_mask_encoded

def create_coco_annotations(binary_dir, json_path):
    """Creates COCO annotations from binary groundtruth TIFF files."""
    coco_annotations = {
        "info": {},
        "licenses": [],
        "images": [],
        "annotations": [],
        "categories": []
    }

    categories = [{"id": 1, "name": "object"}]  # Assuming there's only one category

    coco_annotations["categories"] = categories

    image_id = 1
    annotation_id = 1

    for filename in os.listdir(binary_dir):
        if filename.endswith(".tif"):
            image_path = os.path.join(binary_dir, filename)
            image = np.array(Image.open(image_path))
            height, width = image.shape

            image_info = {
                "id": image_id,
                "file_name": filename,
                "width": width,
                "height": height,
                "license": None,
                "flickr_url": "",
                "coco_url": "",
                "date_captured": None
            }

            coco_annotations["images"].append(image_info)

            # Assuming binary mask contains object
            binary_mask = (image > 0).astype(np.uint8)

            binary_mask_encoded = binary_mask_to_rle(binary_mask)

            annotation = {
                "id": annotation_id,
                "image_id": image_id,
                "category_id": 1,  # Assuming there's only one category
                "segmentation": binary_mask_encoded,
                "area": int(np.sum(binary_mask)),
                "bbox": list(mask.toBbox(binary_mask_encoded)),
                "iscrowd": 0
            }

            coco_annotations["annotations"].append(annotation)

            image_id += 1
            annotation_id += 1

    with open(json_path, 'w') as json_file:
        json.dump(coco_annotations, json_file)

# Example usage
binary_dir = "/media/tova/Memento Mori/inria_dataset/AerialImageDataset/train/gt_patches"
json_path = "/media/tova/Memento Mori/inria_dataset/AerialImageDataset/train/image_patches/_annotations.coco.json"
#binary_dir = "/media/tova/Memento Mori/inria_dataset/AerialImageDataset/valid/gt_patches"
#json_path = "/media/tova/Memento Mori/inria_dataset/AerialImageDataset/valid/image_patches/_annotations.coco.json"

create_coco_annotations(binary_dir, json_path)
